# TEST - Jr. Data Scientist #

## 1. Introduction ##

The task is to explain the relatively complex mathematical concept in a way that a software engineer can comprehend and implement the concept in code.

Particularly, the following formula needs to be explained:

$$L(S,R,U,T,F,G)=\frac{1}{2}||A-S \times_R R \times_U U \times_T T ||^2 + \frac{\lambda_1}{2}||X-TG||^2 + \frac{\lambda_2}{2}||Y-RF||^2 + \frac{\lambda_3}{2}(||S||^2 + ||R||^2 + ||U||^2 + ||T||^2 + ||F||^2 + ||G||^2)$$ 

__Note__. It's important to understand that all the variables in the right-hand side of the equation are 
$n$-dimensional arrays (tensors, matrices and vectors) with $ n \leq 3 $.

There also special math symbols used in the formula:
* $|| \cdot ||$ - is so called "norm". Intuitively, in terms of geometry, it may be considered as the _lenght_ of the vector (do you remember Pythagoras theorem?). With regards to $n$-dim array - we just scale the idea of Euclidean length - it is squre root of sum of squares af all the elements of the array. We will come back to it in the implementation part.
* $S \times_R R$ - is "cross product" of tensors (see [Wiki](https://en.wikipedia.org/wiki/Matrix_multiplication) or [this simple video](https://www.khanacademy.org/math/precalculus/precalc-matrices/multiplying-matrices-by-matrices/v/matrix-multiplication-intro) from Khan Academy). But in our case it is not simple multiplication but tensor-matrix (3D-array by 2D array) multiplication in which each 2D "slice" of the 3D array is multiplied dy 2D array according to matrix multiplication rule.

## 2. Explanation of the formula ##

### a) Objective of the formula ###
The aforementioned formula is so called "loss function" which, in general case, represents quality of output of the prediction model. In simple words, the value returned by this function tells us how the estimated (by predictive model) value far from true value. Having value of loss fuction, one can further update parameters of the model `{S, R, U, T, F, G}` to make it smaller, but this is a separate task called optimization which we will not touch in this document.

In this particular domain this function is used to find optimum travel time taking into account many road grid related features which we will see later.

As it may be seen, the right-hand side of the equation consists of 4 terms. For example, the first one computes the difference (or _distance_) between our, let's say measured, 3D array $A$ and recovered from database archived 3D array $S \times_R R \times_U U \times_T T$. Why it is so complicated? The reason is that this stored 3D array may be very sparse (we will see why in the next subsection) and is stored by set of more dense arrays `{S, R, U, T}`.


### b) Inputs and outputs ###

Let's look at the inputs that our function needs to accept.
__Note__. All inputs are either arrays or floating point values as well as all values in the arrays have to be also `float32` or `float64` (depending on the needed accuracy) data type, hence we will only consider shapes (here and later we mean shape in terms of NumPy) of the input data below.

1. $A$ is 3D array of shape `(N, M, L)` and its values represent time cost of traveling $n^{th}$ road segment by $m^{th}$ driver within $l^{th}$ time slot where $n, m, l$ are elements of respective dimensions indicated by the same big letters.

2. The following four inputs `{S, R, U, T}` are result of  decomposition of stored version of $A$ with some special algorithms which we will not consider here. Basically, their product recovers original version of $A$ but the components require much less space to store as $A$ is sparse due to domain specific reasons. Let's look at the shapes of these four elements:
  * $S$ has the same shape of $A$ as it is basis for its recovering
  * $R$ has shape of `(N, d_R)`
  * $U$ has shape of `(M, d_U)`
  * $T$ has shape of `(L, d_T)`

3. $F$ has shape of `(d_R, Q)` and represents geographical features. Q denotes dimension of these features.
4. $G$ has shape of `(d_T, P)` and represents grid on which road network is divided. P denotes the number of the grids.
5. ${\lambda_1, \lambda_2, \lambda_3}$ are the "tuning" coefficients that are used to change the contribution of each term in the value of loss function. Have to be `float` numbers. 

You may see two other variables in the formula: $X$ and $Y$. They are not inputs but calculated by other decomposition algorithm from $A$ and have shapes of `(L, P)` and `(N, Q)` respectively.

The __output__ of the function should have either `float32` or `float64` data type, depending on needed accuracy again.

### c) Checking of the inputs ###

The following checks of inputs must be done:

1. The shapes of the input arrays (which also need to be NumPy arrays in our implementation) are in compliance with par. 2b. This is very important because, for example, multiplication of matrices requires number of columns of first matrix be equal number of rows of the first.
2. The datatypes of the values of the arrays and lambdas must be `float`.
3. The values should not be negative which is determined by the domain.

### d) Task for the programmer ###

The sofware engineer is supposed to implement function that:
1. Written in the programming language of the company's product
2. Takes all the inputs enumerated in par. 2b
3. Performs input checks enumerated in par. 2c
4. Computes the loss according to the formula with maximum efficiency and utilization of hardware resources 
5. Performs feasibility check of the computed value (at least that it is positive and `float`)
6. All the code is covered by tests
7. Integrates in the company's product seamlessly



## 3. Implementation ##

In this section we will implement and test the function for computing the loss function in Python programming language using NumPy library (as  it has convenient API and optimized for vectorized computation).

First of all, lets import NumPy and construct our toy-data:

In [1]:
import numpy as np

np.random.seed(42) # setting random seed for repeatability of results

# define dimensions
N, M, L = (10, 6, 8)
d_R, d_U, d_T = (5, 4, 4)
Q, P = (6, 7)

#construct our data
A = np.random.rand(N, M, L)
S = np.random.rand(d_R, d_U, d_T)
R = np.random.rand(N, d_R)
U = np.random.rand(M, d_U)
T = np.random.rand(L, d_T)
F = np.random.rand(d_R, Q)
G = np.random.rand(d_T, P)
l1, l2, l3 = np.random.rand(3) # lambdas

X = np.random.rand(L, P) # we initialize X and Y assuming that they are calculated
Y = np.random.rand(N, Q) # with separate function from A

Let's define or function

In [2]:
def loss(A, S, R, U, T, F, G, l1, l2, l3, X, Y):
    '''X, Y shouldn't be input parameters of the function (here - just for prototype)'''
    
    '''TODO(for programmer): Perform all the input checks (for programmer) '''
    
    term1 = 0.5 * np.linalg.norm(A - np.tensordot(  
        np.tensordot(
            np.tensordot(
                S, R, axes=([0],[1])), U, 
            axes=([1],[1])), T, 
        axes=([0],[1])))
    
    # numpy.tensordot performs multiplication of tensor by matrix across axes
    
    '''TODO(for programmer): Assure that shapes match '''
        
    term2 = 0.5 * l1 * np.linalg.norm(X - np.dot(T, G))
    
    term3 = 0.5 * l2 * np.linalg.norm(Y - np.dot(R, F))
    
    term4 = 0.5 * l3 * (np.linalg.norm(S) + 
                       np.linalg.norm(R) +
                       np.linalg.norm(U) +
                       np.linalg.norm(T) +
                       np.linalg.norm(F) +
                       np.linalg.norm(G))
    
    '''TODO(for programmer): Assure that output value is feasible '''
    
    return term1 + term2 + term3 + term4
    

And finally let's test it with our randomly generated data

In [3]:
loss(A, S, R, U, T, F, G, l1, l2, l3, X, Y)

48.087645444389523

Looks like it works!

## 4. Conclusion ##
Of course the code needs to be checked which requires extensive testing but the goal of this document is to show and explain how the relatively complex math concept may be implemented in code. After testing, the code may be rewritten in any other programming language and used in production in order to create value for customers with intelligent solutions.

## 5. References ##

1. Y. Wang, Y. Zheng, and Y. Xue. 2014. Travel Time Estimation of a Path using Sparse Trajectories. In Proceedings of the 20th ACM SIGKDD International Conference on Knowledge Discovery and Data Mining. ACM, 25-34. 
2. Zheng, Yu. "Trajectory data mining: an overview." ACM Transactions on Intelligent Systems and Technology (TIST) 6.3 (2015): 29.